In [ ]:
from sklearn.cluster import DBSCAN
import pandas as pd
import numpy as np

In [ ]:
class TCluster:
    """
    Class that implements the T-Cluster algorithm. It uses the DBSCAN algorithm to cluster the points and then
    uses the trie to predict the words that the user is gaze-typing.
    """

    def __init__(self, eps: float=0.1, min_samples: int=5):
        """
        Constructor for the TCluster class.
        """

        # DBSCAN parameters
        self.eps: float = eps
        self.min_samples: int = min_samples
        self.model: DBSCAN = DBSCAN(eps=eps, min_samples=min_samples)

        # Sets up the data parameters for later
        self.X: pd.DataFrame = None
        self.labels_: list = None
        



    def fit(self, X: pd.DataFrame, verbose: bool=False):
        """
        Fits the model to the data.

        @params:
        X: pd.DataFrame - The data to fit the model to.
        verbose: bool - Whether to print the labels and core samples.
        """

        self.X = X

        # Fit the DBSCAN model to the data
        self.model.fit(X)

        self.labels_ = self.model.labels_
        self.X['label'] = self.labels_

        self._filter_labels()

        if verbose:
            print(f'{len(self.model.labels_)} Labels:', self.model.labels_)
            print('Core samples:', self.model.core_sample_indices_)

    def _filter_labels(self):
        """
        Filters the labels that are -1 and the labels that are 2 IQRs away from the median size of the clusters.
        """
        
        # Noise points
        self.X = self.X[self.X['label'] != -1]

        # Don't keep the labels that the amount of points are 2 IQRs away from the median
        Q1 = self.X['label'].value_counts().quantile(0.25)
        Q3 = self.X['label'].value_counts().quantile(0.75)
        IQR = Q3 - Q1

        self.X = self.X[self.X['label'].map(self.X['label'].value_counts()) > Q1 - 1.5 * IQR]

        # Update the labels
        self.labels_ = self.X['label'].tolist()

In [ ]:
# TODO
center = ...
radius = ...
outerRadius = ...
points =  ... #exactly the gaze_points field from the json request

# Filters the points that are outside the circle or in the inner part
points = [(point['x'], point['y'], point['z']) for point in points if ((point['x'] - center[0])**2 + (point['y'] - center[1])**2 > radius**2 and 
                                                                           (point['x'] - center[0])**2 + (point['y'] - center[1])**2 < outerRadius**2)]

# Create the dataframe
X = pd.DataFrame(points, columns=['x', 'y', 'time'])

In [ ]:
model = TCluster()

# Do the clustering
model.fit(X)

# Get the clustered data with the labels
X = model.X

In [ ]:
# Calculate the position of the centroid of each cluster
centroids = X.groupby('label')[['x', 'y']].mean()

# **Plotting**

In [ ]:
#TODO